In [92]:
from __future__ import print_function

import pandas as pd

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

In [96]:
raw_data = pd.read_csv('C:\Users\pedro.castanha\Downloads\ML_Gabinete_Digital.csv',
                       error_bad_lines=False,
                       sep='\t',
                       encoding='utf_8')

stop = set(stopwords.words('portuguese'))

x_vectorized = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=stop)
x_vectorized.fit(raw_data.Text)
x_train = x_vectorized.transform(raw_data.Text)
y_train = raw_data.Class

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    words = []
    for item in tokens:
        words.append(' '.join(item))
    return words

In [ ]:
corpus = []
for text in raw_data.Text:
    text = text.lower()
    text=tokenize(text)
    corpus.append(text)

#x_train_line = sequence.pad_sequences(raw_data.Text, maxlen=maxlen)

In [100]:
raw_data.Text[0].encode()

UnicodeEncodeError: 'ascii' codec can't encode character u'\xf3' in position 2: ordinal not in range(128)

In [99]:
toke = Tokenizer()
inpt = toke.texts_to_sequences(raw_data.Text[0])
inpt

TypeError: character mapping must return integer, None or unicode

In [ ]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Building model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Model complete...')

In [ ]:
# Running the model
print('Running...')
model.fit(x_train, y_train, verbose=2,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

print('Complete')

In [ ]:
model.summary()

In [ ]:
model.evaluate(x_test,y_test,verbose=0)

In [ ]:
model.metrics_names